In [1]:
from mudpy import fakequakes,runslip,forward,viewFQ
import numpy as np
from obspy.core import UTCDateTime,read
from shutil import copy

In [16]:
########                            GLOBALS                             ########

home = '/hdd/rc_fq/summer23/' # set up for Valdivia
project_name = 'june30'
run_name = 'june30'

In [39]:
##############             What do you want to do??           ##################

init = 1
make_ruptures = 1
make_GFs = 1
make_synthetics = 1
make_waveforms = 1
make_hf_waveforms = 0
match_filter = 0
make_statics = 0
# Things that only need to be done once
load_distances = 0 # for make_ruptures. Set to 0 on first run and 1 after
G_from_file = 0 # for make_waveforms. Set to 0 on first run and 1 after

In [40]:
##############                 Run parameters                ##################

# Runtime parameters 
ncpus = 50                                        # How many CPUS you want to use for parallelization (needs to be at least 2)
Nrealizations = 100                                # Number of fake ruptures to generate per magnitude bin - ncups overrides this?
hot_start = 0

# File parameters
model_name = 'mojave.mod'
# model_name = 'ridgecrest.mod'                      # Velocity model file name
fault_name = 'ridgecrest_m7_fault3.fault'                    # Fault model name
mean_slip_name = None                            # Set to path of .rupt file if patterning synthetic runs after a mean rupture model
# run_name = 'rcrest_m7'                            # Base name of each synthetic run (i.e. mentawai.000000, mentawai.000001, etc...)
rupture_list = 'ruptures.list'                   # Name of list of ruptures that are used to generate waveforms.  'ruptures.list' uses the full list of ruptures FakeQuakes creates. If you create file with a sublist of ruptures, use that file name.
distances_name = 'original'                      # Name of matrix with estimated distances between subfaults i and j for every subfault pair                              # This should be zero the first time you run FakeQuakes with your fault model.

# Source parameters
UTM_zone = '11S'                                 # UTM_zone for rupture region 
time_epi = UTCDateTime('2019-07-06T03:19:53.040')   # Origin time of event (can set to any time, as long as it's not in the future)
# target_Mw = np.array([4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5])                      # Desired magnitude(s), can either be one value or an array
target_Mw_flip = np.arange(4.3,7.6,0.1) # goes from 4.3 to 7.5, 100 ruptures per 0.1 mag bin for 3300 total
target_Mw = np.flip(target_Mw_flip)
hypocenter = None                                # Coordinates of subfault closest to desired hypocenter, or set to None for random
force_hypocenter = False                         # Set to True if hypocenter specified
rake = 180                                        # Average rake for subfaults
scaling_law = 'S'                                # Type of rupture: T for thrust, S for strike-slip, N for normal
force_magnitude = False                          # Set to True if you want the rupture magnitude to equal the exact target magnitude
force_area = False                               # Set to True if you want the ruptures to fill the whole fault model

# Correlation function parameters
hurst = 0.4                                      # Hurst exponent form Melgar and Hayes 2019
Ldip = 'auto'                                    # Correlation length scaling: 'auto' uses Melgar and Hayes 2019, 'MB2002' uses Mai and Beroza 2002
Lstrike = 'auto'                                 # Same as above
slip_standard_deviation = 0.9                    # Standard deviation for slip statistics: Keep this at 0.9
lognormal = True                                 # Keep this as True to solve the problem of some negative slip subfaults that are produced

# Rupture propagation parameters
rise_time_depths = [1,2]                       # Transition depths for rise time scaling (if slip shallower than first index, rise times are twice as long as calculated)
max_slip = 40                                    # Maximum sip (m) allowed in the model
max_slip_rule = False                            # If true, uses a magntidude-depence for max slip
shear_wave_fraction_shallow = 0.5                        # 0.8 is a standard value (Mai and Beroza 2002)
shear_wave_fraction_deep = 0.8
source_time_function = 'dreger'                  # options are 'triangle' or 'cosine' or 'dreger'
stf_falloff_rate = 4                             # Only affects Dreger STF, 4-8 are reasonable values
num_modes = 72                                   # Number of modes in K-L expansion
slab_name = None                                 # Slab 2.0 Ascii file for 3D geometry, set to None for simple 2D geometry
mesh_name = None                                 # GMSH output file for 3D geometry, set to None for simple 2D geometry

# Green's Functions parameters
GF_list = 'rc_gflist.gflist'                            # Stations file name
G_name = 'rc'                                    # Basename you want for the Green's functions matrices                           # This should be 1 to make the synthetics                                # This should be zero the first time you run FakeQuakes with your fault model and stations.

# fk parameters
# used to solve wave equation in frequency domain 
dk = 0.1 ; pmin = 0 ; pmax = 1 ; kmax = 20             # Should be set to 0.1, 0, 1, 20
custom_stf = None                                # Assumes specified source time function above if set to None

# Low frequency waveform parameters
dt = 1.0                                         # Sampling interval of LF data 
NFFT = 512                                       # Number of samples in LF waveforms (should be in powers of 2)
# dt*NFFT  =  length of low-frequency dispalcement record
# want this value to be close to duration (length of high-frequency record)

# High frequency waveform parameters
stress_parameter = 50                            # Stress drop measured in bars (standard value is 50)
moho_depth_in_km = 30.0                          # Average depth to Moho in this region 
Pwave = True                                     # Calculates P-waves as well as S-waves if set to True, else just S-Waves
kappa = None                                     # Station kappa values: Options are GF_list for station-specific kappa, a singular value for all stations, or the default 0.04s for every station if set to None
hf_dt = 0.01                                     # Sampling interval of HF data
duration = 250                                   # Duration (in seconds) of HF record

high_stress_depth = 30                           # Doesn't do anything, but still shows up as a parameter. Set to whatever you want. 

# Match filter parameters
zero_phase = True                                # If True, filters waveforms twice to remove phase, else filters once
order = 4                                        # Number of poles for filters
fcorner = 0.998                                  # Corner frequency at which to filter waveforms (needs to be between 0 and the Nyquist frequency)


In [41]:
# Set up project folder

if init == 1:
    
    fakequakes.init(home,project_name)

    # Copy files into the project folder

    copy('/home/sdybing/rc_fq/mojave.mod', '/' + home + '/' + project_name + '/structure')
    copy('/home/sdybing/rc_fq/ridgecrest_m7_fault3.fault', '/' + home + '/' + project_name + '/data/model_info')
    copy('/home/sdybing/rc_fq/rc_gflist.gflist', '/' + home + '/' + project_name + '/data/station_info')
    


Project directory exists, clobber (y/n)?y
This will delete everything in this project directory, so, take a minute, think about it: clobber (y/n)?y


In [42]:
# Generate rupture models

if make_ruptures == 1:
    
    fakequakes.generate_ruptures(home,project_name,run_name,fault_name,slab_name,mesh_name,load_distances,
        distances_name,UTM_zone,target_Mw,model_name,hurst,Ldip,Lstrike,num_modes,Nrealizations,rake,
        rise_time_depths,time_epi,max_slip,source_time_function,lognormal,slip_standard_deviation,scaling_law,
        ncpus,mean_slip_name=mean_slip_name,force_magnitude=force_magnitude,force_area=force_area,
        hypocenter=hypocenter,force_hypocenter=force_hypocenter,shear_wave_fraction_shallow=shear_wave_fraction_shallow,
        shear_wave_fraction_deep=shear_wave_fraction_deep,max_slip_rule=max_slip_rule)
    


Building obspy.taup model for '/hdd/rc_fq/summer23/june30/structure/mojave.nd' ...
filename = /hdd/rc_fq/summer23/june30/structure/mojave.nd
Done reading velocity model.
Radius of model . is 6371.0
Using parameters provided in TauP_config.ini (or defaults if not) to call SlownessModel...
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 kilometers
taup.create.max_range_interval = 0.04363323129985824 degrees
taup.create.max_interp_error = 0.05 seconds
taup.create.allow_inner_core_s = True
Slow model  959 P layers,1223 S layers
Done calculating Tau branches.
Done Saving /hdd/rc_fq/summer23/june30/structure/mojave.npz
Method run is done, but not necessarily successful.
MPI: Starting 100 FakeQuakes Rupture Generations on  50 CPUs


No protocol specified


Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 10 of 54
... working on subfault 20 of 54
Getting inter-fault distances
... working on subfault 30 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 0 of 54
... working on subfault 40 of 54
Getting inter-fault distances
... working on subfault 0 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 10 of 54
... working on subfault 10 of 54
... working on subfault 10 of 54
Getting inter-fault distances
... working on subfault 0 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 50 of 54
... working on subfault 10 of 54
Getting inter-fault distances
... working on subfault 0 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 20 of 54
... working on subfault 10 of 54
... working on subfault 10 of 54
... working on subfault 20 of 54
... working o

Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 10 of 54
Getting inter-fault distances
... working on subfault 0 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 50 of 54
... working on subfault 30 of 54
... working on subfault 30 of 54
... working on subfault 50 of 54
... working on subfault 50 of 54
... working on subfault 30 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 10 of 54
... working on subfault 10 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 10 of 54
... working on subfault 20 of 54
... working on subfault 10 of 54
... working on subfault 40 of 54
... working on subfault 20 of 54
... working on subfault 10 of 54
Getting inter-fault distances
... working on subfault 0 of 54
... working on subfault 10 of 54
... working on subfault 20 of 54
... working on subfault 30 of 54
... working on subfault 20 of 54
.

... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... max slip condition violated d

... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalcu

... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalculating...
... ... ... max slip condition violated due to force_magnitude=True, recalculating...
... ... ... improper eigenvalues, recalcu

... ... working on ruptures 1350 to 1399 of 3300
... Calculating ruptures for target magnitude Mw = 6.099999999999993
... ... working on ruptures 1400 to 1449 of 3300
... ... working on ruptures 1450 to 1499 of 3300
... Calculating ruptures for target magnitude Mw = 5.999999999999994
... ... working on ruptures 1500 to 1549 of 3300
... ... working on ruptures 1550 to 1599 of 3300
... Calculating ruptures for target magnitude Mw = 5.899999999999994
... ... working on ruptures 1600 to 1649 of 3300
... ... working on ruptures 1650 to 1699 of 3300
... Calculating ruptures for target magnitude Mw = 5.7999999999999945
... ... working on ruptures 1700 to 1749 of 3300
... ... working on ruptures 1750 to 1799 of 3300
... Calculating ruptures for target magnitude Mw = 5.699999999999995
... ... working on ruptures 1800 to 1849 of 3300
... ... working on ruptures 1850 to 1899 of 3300
... Calculating ruptures for target magnitude Mw = 5.599999999999995
... ... working on ruptures 1900 to 1949 of 33

In [43]:
# Make Green's functions

if make_GFs == 1 or make_synthetics == 1:
    
    runslip.inversionGFs(home,project_name,GF_list,None,fault_name,model_name,
        dt,None,NFFT,None,make_GFs,make_synthetics,dk,pmin,
        pmax,kmax,0,time_epi,hot_start,ncpus,custom_stf,impulse=True) 
    



Cutting back to 27 cpus for 54 subfaults
Displacememnt GFs requested...
MPI: Starting GFs computation on 27 CPUs

mpiexec -n 27 python /home/sdybing/MudPy/src/python/mudpy/parallel.py run_parallel_green /hdd/rc_fq/summer23/ june30 /hdd/rc_fq/summer23/june30/data/station_info/temp.sta mojave.mod 1.0 512 0 0.1 0 1 20 False False


No protocol specified


Running all processes with:
        home = /hdd/rc_fq/summer23/
        project_name = june30
        station_file = /hdd/rc_fq/summer23/june30/data/station_info/temp.sta
        model_name = mojave.mod
        static = 0
        tsunami = False
        dt = 1.000
        NFFT = 512
        dk = 0.100
        pmin = 0.000
        pmax = 1.000
        kmax = 20.000
        insar = False
        
MPI: processor # 8 is now working on subfault 9 ( 1 / 2 )
MPI: processor # 4 is now working on subfault 5 ( 1 / 2 )
MPI: processor # 26 is now working on subfault 27 ( 1 / 2 )
MPI: processor # 15 is now working on subfault 16 ( 1 / 2 )
MPI: processor # 17 is now working on subfault 18 ( 1 / 2 )
MPI: processor # 16 is now working on subfault 17 ( 1 / 2 )
MPI: processor # 24 is now working on subfault 25 ( 1 / 2 )
MPI: processor # 22 is now working on subfault 23 ( 1 / 2 )
MPI: processor # 11 is now working on subfault 12 ( 1 / 2 )
MPI: processor # 23 is now working on subfault 24 ( 1 / 2 )
MPI: p

No protocol specified


Running all processes with:
        home = /hdd/rc_fq/summer23/
        project_name = june30
        station_file = /hdd/rc_fq/summer23/june30/data/station_info/temp.sta
        model_name = mojave.mod
        integrate = 1
        static = 0
        tsunami = False
        time_epi = 2019-07-06T03:19:53.040000Z
        beta = 0
        custom_stf = None
        impulse = True
        insar = False
        okada = False
        mu = 4.50e+10
        
Processor 0 is working on subfault 1 and 278 stations 
Processor 19 is working on subfault 20 and 278 stations 
Processor 11 is working on subfault 12 and 278 stations 
Processor 5 is working on subfault 6 and 278 stations 
Processor 23 is working on subfault 24 and 278 stations 
Processor 13 is working on subfault 14 and 278 stations 
Processor 8 is working on subfault 9 and 278 stations 
Processor 16 is working on subfault 17 and 278 stations 
Processor 3 is working on subfault 4 and 278 stations 
Processor 20 is working on subfault 21 

In [44]:
# # Make low frequency displacement waveforms

if make_waveforms == 1:
    
    forward.waveforms_fakequakes(home,project_name,fault_name,rupture_list,GF_list, # need to shorten path again
        model_name,run_name,dt,NFFT,G_from_file,G_name,source_time_function,
        stf_falloff_rate,ncpus=ncpus)
    
# See some of the waveforms

# stas = np.genfromtxt('/Users/sydneydybing/RC_FQ/flt3_tst_rnge/data/station_info/RC_gflist_short.gflist', usecols=0, dtype=str)
# rupt = '/rcrest_M6.000000'

# for sta in stas:
    
#     st = read(f'{home}{project_name}/output/waveforms' + rupt + '/' + sta + '.LYE.sac')
#     st.plot()




Solving for kinematic problem(s)
... loading all synthetics into memory
... ... reading green functions for station #1 of 278
... ... reading green functions for station #2 of 278
... ... reading green functions for station #3 of 278
... ... reading green functions for station #4 of 278
... ... reading green functions for station #5 of 278
... ... reading green functions for station #6 of 278
... ... reading green functions for station #7 of 278
... ... reading green functions for station #8 of 278
... ... reading green functions for station #9 of 278
... ... reading green functions for station #10 of 278
... ... reading green functions for station #11 of 278
... ... reading green functions for station #12 of 278
... ... reading green functions for station #13 of 278
... ... reading green functions for station #14 of 278
... ... reading green functions for station #15 of 278
... ... reading green functions for station #16 of 278
... ... reading green functions for station #17 of 278
..

... ... reading green functions for station #148 of 278
... ... reading green functions for station #149 of 278
... ... reading green functions for station #150 of 278
... ... reading green functions for station #151 of 278
... ... reading green functions for station #152 of 278
... ... reading green functions for station #153 of 278
... ... reading green functions for station #154 of 278
... ... reading green functions for station #155 of 278
... ... reading green functions for station #156 of 278
... ... reading green functions for station #157 of 278
... ... reading green functions for station #158 of 278
... ... reading green functions for station #159 of 278
... ... reading green functions for station #160 of 278
... ... reading green functions for station #161 of 278
... ... reading green functions for station #162 of 278
... ... reading green functions for station #163 of 278
... ... reading green functions for station #164 of 278
... ... reading green functions for station #165

... ... slip rate convolutions completed: wall time 13.0s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 4 of 3300: june30.000004.rupt
... ... slip rate convolutions completed: wall time 16.4s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 5 of 3300: june30.000005.rupt
... ... slip rate convolutions completed: wall time 17.3s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 6 of 3300: june30.000006.rupt
... ... slip rate convolutions completed: wall time 2.7s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 7 of 3300: june30.000007.rupt
... ... slip rate convolutions completed: wall time 14.8s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 8 of 3300: june30.000008.rupt
... ... slip rate convolutions completed: wall time 11.9s
... ... 

... ... slip rate convolutions completed: wall time 15.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 48 of 3300: june30.000048.rupt
... ... slip rate convolutions completed: wall time 14.8s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 49 of 3300: june30.000049.rupt
... ... slip rate convolutions completed: wall time 24.4s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 50 of 3300: june30.000050.rupt
... ... slip rate convolutions completed: wall time 14.7s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 51 of 3300: june30.000051.rupt
... ... slip rate convolutions completed: wall time 20.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 52 of 3300: june30.000052.rupt
... ... slip rate convolutions completed: wall time 16.7s
..

... ... slip rate convolutions completed: wall time 14.7s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 92 of 3300: june30.000092.rupt
... ... slip rate convolutions completed: wall time 9.7s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 93 of 3300: june30.000093.rupt
... ... slip rate convolutions completed: wall time 22.3s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 94 of 3300: june30.000094.rupt
... ... slip rate convolutions completed: wall time 13.0s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 95 of 3300: june30.000095.rupt
... ... slip rate convolutions completed: wall time 15.8s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 96 of 3300: june30.000096.rupt
... ... slip rate convolutions completed: wall time 8.9s
... 

...

... ... finished with this rupture
... solving for source 3100 of 3300: june30.003100.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3101 of 3300: june30.003101.rupt
... ... slip rate convolutions completed: wall time 0.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3102 of 3300: june30.003102.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3103 of 3300: june30.003103.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3104 of 3300: june30.003104.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for sour

... ... finished with this rupture
... solving for source 3144 of 3300: june30.003144.rupt
... ... slip rate convolutions completed: wall time 0.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3145 of 3300: june30.003145.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3146 of 3300: june30.003146.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3147 of 3300: june30.003147.rupt
... ... slip rate convolutions completed: wall time 0.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3148 of 3300: june30.003148.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for sour

... ... finished with this rupture
... solving for source 3188 of 3300: june30.003188.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3189 of 3300: june30.003189.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3190 of 3300: june30.003190.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3191 of 3300: june30.003191.rupt
... ... slip rate convolutions completed: wall time 0.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3192 of 3300: june30.003192.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for sour

... ... finished with this rupture
... solving for source 3232 of 3300: june30.003232.rupt
... ... slip rate convolutions completed: wall time 0.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3233 of 3300: june30.003233.rupt
... ... slip rate convolutions completed: wall time 0.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3234 of 3300: june30.003234.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3235 of 3300: june30.003235.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3236 of 3300: june30.003236.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for sour

... ... finished with this rupture
... solving for source 3276 of 3300: june30.003276.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3277 of 3300: june30.003277.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3278 of 3300: june30.003278.rupt
... ... slip rate convolutions completed: wall time 0.1s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3279 of 3300: june30.003279.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for source 3280 of 3300: june30.003280.rupt
... ... slip rate convolutions completed: wall time 0.2s
... ... solve for waveforms (ncpus = 50)
... ... finished with this rupture
... solving for sour